In [1]:
from urllib.request import urlopen
import re
import numpy as np
import pandas as pd

# 1. Membuat Tabel Peluang Frekuensi Bahasa Sunda

In [2]:
# Membuka file .txt berisi teks asli dan menyimpannya sebagai string pada variabel "data"
data = urlopen('https://raw.githubusercontent.com/ilmaaliyaf/kriptanalisis-vigenere/main/sunda_example.txt').read().decode('utf-8')
print(data[:100])

Kampanyeu... kampanyeu... kampanyeu!
Ngawawarkeun paniatan hayang boga kahadéan keur balarea....cen


In [3]:
# Membersihkan "data" dari semua karakter (termasuk whitespace) kecuali alfabet serta mengubah tiap karakter menjadi lowercase
data = re.sub('[\W_]+', '', data).lower()
data = re.sub('έ', 'é', data)
data = re.sub('\d+', '', data)
print(data)

kampanyeukampanyeukampanyeungawawarkeunpaniatanhayangbogakahadéankeurbalareacenahnahamakékududiwawarkeunapanbaturacanwawuheunkauingcanwanohkanadedegpangadegcanapalkanakaahlianuingcanapalkanapaniatanuingkatingalbabalapananapaalusaluspasangpetapahadéhadényieunuarpangajakjeungpagedégedépangbibitaarikitunamahkahartiapannukampanyeutéhlainsaurangduaurangpuluhanmalahratusanpadapadahayangkapilihkubalaréapalayjenengandegléngdinakorsigoyangdigedongsigrongdisenayankajurungkungagedurnakarephayangnyaitnasibngangkatdarajatdulursalemburbarayasanagaraatuhdinapraknakampanyeutéhlobapisananupohokanatatakramajigasakarepingsunteunolihwanciteumalirélahanpanayanugeusderengdengmiheulaanbatursaméméhpiriwitditiupkukpujeungpnwaslunumidangdinakoranjeungmajalahnambahanpangasilankapamedalnanunarangkodbaédinatangkalkaijeungtihanglistriksapanjangjalantéhteumalirékanalaranganpamaréntahkotaatributkampanyeuulahngarurujitkaéndahankotaaripokpokananajangjinaésatuaramispisanjiganateukaémutnahaétajangjitéhgeusdiijirbakalbisa

In [4]:
#Menghitung Frekuensi Kemunculan Huruf pada Text
counts_dict = {}
for c in list(data):
  if c.upper() not in counts_dict:
    counts_dict[c.upper()] = 0
  counts_dict[c.upper()] += 1

# menyortir kamus
counts_dict = dict(sorted(counts_dict.items()))

In [5]:
#Mengubah dari dict ke dataframe
dffrek=pd.DataFrame.from_dict(counts_dict,orient='index', columns=['Frekuensi'])

In [6]:
#Menghitung Peluang Dari Kemunculan Huruf
total = dffrek['Frekuensi'].sum()
peluang=[]
peluang=(dffrek['Frekuensi']/total)

In [7]:
#Menggabungkan Column Frekuensi dan Peluang Kemunculan Huruf
dffrek.rename(columns={'Frekuensi':'Frekuensi '}, inplace=True)
gabfrekpel=pd.concat([dffrek, peluang], axis=1)
gabfrekpel=gabfrekpel.sort_index()
gabfrekpel.rename(columns={'Frekuensi':'Peluang'}, inplace=True)
print(gabfrekpel)
print(gabfrekpel['Peluang'].sum())

   Frekuensi    Peluang
A       25290  0.198359
B        2946  0.023107
C         894  0.007012
D        3820  0.029962
E        6526  0.051186
F         111  0.000871
G        6649  0.052151
H        3840  0.030119
I        8558  0.067124
J        1871  0.014675
K        6409  0.050268
L        4067  0.031899
M        3981  0.031225
N       14747  0.115666
O        2811  0.022048
P        3546  0.027813
Q          21  0.000165
R        5541  0.043460
S        4606  0.036127
T        5610  0.044001
U       10217  0.080136
V          40  0.000314
W        1245  0.009765
X          10  0.000078
Y        1485  0.011647
Z          23  0.000180
É        2632  0.020644
1.0


In [8]:
# Mengubah nilai-nilai peluang menjadi array 1d
prob = np.array(list(peluang.values))
total_alph = len(prob)
prob

array([1.98359164e-01, 2.31066073e-02, 7.01198469e-03, 2.99617243e-02,
       5.11859196e-02, 8.70615549e-04, 5.21506557e-02, 3.01185920e-02,
       6.71236745e-02, 1.46749702e-02, 5.02682437e-02, 3.18990400e-02,
       3.12245090e-02, 1.15666374e-01, 2.20477505e-02, 2.78126373e-02,
       1.64711050e-04, 4.34601870e-02, 3.61266236e-02, 4.40013804e-02,
       8.01358474e-02, 3.13735333e-04, 9.76501224e-03, 7.84338332e-05,
       1.16474242e-02, 1.80397816e-04, 2.06437849e-02])

In [9]:
# Menghitung peluang kemunculan huruf
P = np.sum(np.square(prob))
P

0.08316258094591088

# 2. Enkripsi Kata-asal Berbahasa Sunda

In [10]:
# Mendefinisikan fungsi penghitung frekuensi huruf
def freq(s):
    ''' Menghitung frekuensi huruf pada string s '''
    fr = {i : s.count(i) for i in set(s)}
    for c in counts_dict: # kamus dari section 1
        if c not in fr:
            fr[c] = 0
    return dict(sorted(fr.items()))
    
# Fungsi pengubah huruf menjadi angka
def ConvertToNum(string):
    '''input s berupa string, output berupa list integer'''
    return [26 if x=='é' else ord(x)-97 for x in string]

# Fungsi pengubah angka menjadi huruf
def ConvertToText(num_list):
    '''input n berupa list integer, output berupa string'''
    return ''.join([chr(ord('é'.upper())) if x==26 else chr(x+65) for x in num_list])

# Fungsi enkripsi sandi Vigenre
def EncryptVigenere(plain_text, key_text):
    '''input plain_text dan key berupa string, output berupa string'''
    plain_text_num = ConvertToNum(plain_text)
    key_num = ConvertToNum(key_text)
    m = len(key_num)
    cipher_text_num = []
    for i in range(len(plain_text_num)):
        cipher_text_num.append((plain_text_num[i] + key_num[i%m]) % total_alph)
    return ConvertToText(cipher_text_num)

# Fungsi dekripsi sandi Vigenere
def DecryptVigenere(cipher_text, key_text):
    '''input cipher_text dan key berupa string, output berupa string'''
    cipher_text_num = ConvertToNum(cipher_text.lower())
    key_num = ConvertToNum(key_text.lower())
    m = len(key_num)
    plain_text_num = []
    for i in range(len(cipher_text_num)):
        plain_text_num.append((cipher_text_num[i] - key_num[i%m]) % total_alph)
    return ConvertToText(plain_text_num).lower()

In [11]:
# Mendefinisikan kata-asal berbahasa sunda
plaintext = '''Harta pada naréangan harti pada nyararungsi sabab duanana guna
harti bisa méré bukti harta pon kitu deui bisa ngabul nu di maksud
neudunan sakahayang tapi harta gancang leungit mungguh harti mangfaat dunya akhérat
'''
# Membersihkan kata-asal
plaintext = re.sub('[\W_]+', '', plaintext).lower()
print(plaintext)

hartapadanaréanganhartipadanyararungsisababduananagunahartibisamérébuktihartaponkitudeuibisangabulnudimaksudneudunansakahayangtapihartagancangleungitmungguhhartimangfaatdunyaakhérat


In [12]:
# Mendefinisikan kata kunci
key = 'wayahna'
len_k = len(key)
print(key)

wayahna


In [13]:
# Mengenkripsi kata asal, vigenere chiper, y_i = x_i + k_i
ciphertext = EncryptVigenere(plaintext,key)
print(ciphertext)

CAOTHBAZAKAYMAIGYNONROIMAKNNTAOAYGNBSFSHOAXDRAUNNWGRNHUAMTFBPEAHÉOÉIGKOIEAYFAKOKKPFUZERIIVSWNDAIGLIUAITNKNUANLGDPNYNZNKWHYYHÉGOAMIONROADAUPAIGIEAÉGDTJUUTGPHEAYFIHAKGMNAODRNENAFHXRHF


In [14]:
# Mengecek hasil enkripsi dengan cara dekripsi langsung
print(DecryptVigenere(ciphertext,key))

hartapadanaréanganhartipadanyararungsisababduananagunahartibisamérébuktihartaponkitudeuibisangabulnudimaksudneudunansakahayangtapihartagancangleungitmungguhhartimangfaatdunyaakhérat


# 3. Tes Kasiski

In [15]:
import operator

In [16]:
# Mendata semua kombinasi 3-huruf di ciphertext
tri = []
for i in range(len(ciphertext)-2):
    tri.append(ciphertext[i:i+3])
f_tri = freq(tri)    

# Memfilter kombinasi 3-huruf yang frekuensinya lebih dari 1
sorted_tri = list(sorted(f_tri.items(), key=operator.itemgetter(1)))
multp_tri = [sorted_tri[i] for i in range(len(sorted_tri)) if sorted_tri[i][1] > 1]
print(multp_tri)

# Menghitung selisih posisi untuk tiap 3-huruf
pos = []
diff = []
for i in range(len(multp_tri)):
    pos.append([m.start() for m in re.finditer(multp_tri[i][0], ciphertext)])
    diff.append(pos[i][1] - pos[i][0])
    print(multp_tri[i][0], pos[i], diff[i])

[('AYF', 2), ('EAY', 2), ('IEA', 2), ('NRO', 2), ('ONR', 2), ('AIG', 3)]
AYF [73, 157] 84
EAY [72, 156] 84
IEA [71, 142] 71
NRO [19, 131] 112
ONR [18, 130] 112
AIG [13, 94, 139] 81


In [17]:
# Mencari gcd dari tiap selisih posisi (opsional)

def GCD(a, b):
    ''' mencari faktor persekutuan terbesar antara a dan b'''
    if b == 0:
        return a
    else:
        return GCD(b, a % b)

gcd = GCD(diff[1],diff[2]) 
for i in range(3,len(diff)): 
    gcd=GCD(gcd,diff[i]) 
      
print(GCD(84,112))

28


# 4. Tes Indeks Koinsiden Bersama

In [18]:
def cutter(s,m):
    ''' Memotong string s menjadi m potongan string '''
    n = len(s)
    chunks = [''] * m
    for k in range(m):
        l = list(range(k, n, m))
        chunks[k] = ''.join([s[x] for x in l])
    return chunks

def ic(s):
    ''' Menghitung indeks koinsidensi dari string s'''
    n = len(s)
    fr = freq(s)
    sum = 0
    for i in fr:
        sum += fr[i] * (fr[i]-1)
    return sum/(n*(n-1))

def avg_ic(ciphertext):
    ''' Menebak kemungkinan panjang key
        merata-ratakan indeks koinsiden setiap chunk pada ciphertext_cut
        output berupa panjang key m'''
    m = 0
    ic_avg = 0
    while ic_avg < P:
        m += 1
        ciphertext_cut = cutter(ciphertext,m)
        s = 0
        for i in range(m):
            s += ic(ciphertext_cut[i])
        ic_avg = s/m
        print('m =',m, '| avg ic:',ic_avg)
    return m

def mic(ct,m):
    ''' Menghitung mutual index of coincidence dari string ct dan panjang key m
        Output berupa 2d array matrix MIc'''
    len_ct = len(ct)
    n_ = len_ct/m 
    ct_cut = cutter(ct,m) #memotong ciphertext

    # Menghitung frekuensi huruf di ciphertext_cut dan membuatnya menjadi list
    f = np.zeros([m, total_alph])
    for i in range(m):
        f[i] = np.array(list(freq(ct_cut[i]).values()))

    # membuat matriks Mutual Index Coincidence
    mic_matrix = np.zeros([m, total_alph])
    for j in range(m): # mencari kunci digit ke-j
        for g in range(total_alph): # dilihat peluang koinsiden antara alphabet ke g dan ke i
            sum = 0
            for i in range(total_alph):
                sum += prob[i] * f[j][ (i+g) % total_alph ] # (f/n)*f'
            mic_matrix[j][g] = sum/(n_)
    return mic_matrix

In [19]:
# Menebak panjang key
m = avg_ic(ciphertext)

m = 1 | avg ic: 0.05887047268262738
m = 2 | avg ic: 0.060387427803158145
m = 3 | avg ic: 0.06180420487172363
m = 4 | avg ic: 0.0595740008783487
m = 5 | avg ic: 0.05625911625911626
m = 6 | avg ic: 0.0668149796069707
m = 7 | avg ic: 0.09706959706959706


In [20]:
# Membentuk matriks indeks koinsiden
MIc_matrix = mic(ciphertext,m)
pd.DataFrame(MIc_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,0.029366,0.048137,0.048395,0.034034,0.034767,0.036161,0.033083,0.039884,0.051478,0.031689,0.037379,0.027075,0.036371,0.027631,0.061375,0.052911,0.026046,0.028870,0.022496,0.033953,0.021383,0.042610,0.069093,0.033414,0.033953,0.034547,0.029427
1,0.086659,0.029747,0.022461,0.037996,0.024009,0.022992,0.047102,0.065744,0.039035,0.024351,0.029592,0.018852,0.024575,0.047071,0.059517,0.032129,0.032220,0.034097,0.030256,0.044543,0.055414,0.042526,0.029625,0.035953,0.024891,0.024020,0.040149
2,0.043733,0.030179,0.028589,0.035501,0.066713,0.036313,0.032533,0.028147,0.027819,0.031612,0.052330,0.040450,0.036169,0.030838,0.049577,0.020864,0.023728,0.067391,0.031696,0.034339,0.029564,0.029323,0.030520,0.035460,0.073137,0.024707,0.034294
3,0.107487,0.030147,0.022908,0.020280,0.029095,0.030529,0.029034,0.052219,0.046533,0.035991,0.035476,0.020608,0.022681,0.063235,0.052393,0.030492,0.030046,0.036678,0.034913,0.035441,0.050948,0.033976,0.028658,0.036185,0.032622,0.027242,0.029707
4,0.044313,0.036898,0.030352,0.026029,0.045043,0.028232,0.029980,0.078028,0.045046,0.020583,0.027715,0.043156,0.034267,0.018682,0.054371,0.043085,0.034453,0.030799,0.026774,0.028664,0.047212,0.047209,0.035138,0.027699,0.060268,0.032876,0.028650
5,0.056407,0.039315,0.036798,0.028272,0.026613,0.061464,0.061362,0.033550,0.020071,0.030686,0.018244,0.019799,0.036437,0.099735,0.035518,0.032265,0.020504,0.011818,0.025133,0.037822,0.066835,0.039540,0.038169,0.032006,0.021376,0.031544,0.044240
6,0.107191,0.019387,0.024523,0.031620,0.022342,0.025115,0.037839,0.052286,0.029745,0.030620,0.052496,0.019989,0.019298,0.048560,0.054552,0.022126,0.024503,0.050100,0.030100,0.034797,0.051239,0.031255,0.019263,0.035549,0.035887,0.030409,0.026059


In [21]:
# Mencari nilai maksimum di antara semua MIc untuk dipilih sbg digit k di key
K = np.argmax(MIc_matrix,1)

# Mencetak Huruf terpilih sebagai key beserta MIc-nya
print('Huruf \t MIc')
for i in range(m):
    print(K[i],'\t',MIc_matrix[i,K[i]])
Key = ConvertToText(K)
print('Kunci:\t', Key)

# Dekripsi kata sandi dengan key terpilih
ptext = DecryptVigenere(ciphertext,Key)
print('Kata terdekripsi:\n', ptext)

Huruf 	 MIc
22 	 0.06909279701809301
0 	 0.08665863897105905
24 	 0.0731365594570056
0 	 0.10748685171620162
7 	 0.07802844730130414
13 	 0.09973451230795843
0 	 0.10719109983127625
Kunci:	 WAYAHNA
Kata terdekripsi:
 hartapadanaréanganhartipadanyararungsisababduananagunahartibisamérébuktihartaponkitudeuibisangabulnudimaksudneudunansakahayangtapihartagancangleungitmungguhhartimangfaatdunyaakhérat
